In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
spark=SparkSession.builder.appName('data_processing').getOrCreate()

En fonction de nos besoins, on peut avoir besoin de creer des sous ensembles selon plusieurs conditions soit mettant en place
  - Des filtres
  - Des selections par colone ou données
  - Passer par une clause comme Where

In [38]:
users_csv_file = spark.read.csv("users.csv", header=True, inferSchema=True)


In [34]:
# Utilisation de la méthode Select en specifiant la colone à affcher.
users_csv_file.select(["name","country"]).show()

+-------------------+------------+
|               name|     country|
+-------------------+------------+
|       Adara Mcneil|     Austria|
|     Barry Valencia|      Sweden|
|      Barry Delgado|     Vietnam|
|      Shelley Oneal|      Canada|
|     Wyoming Cherry|South Africa|
|       Lani Holcomb|     Germany|
|       Cedric Leach|     Austria|
|          Edan Wood|       Chile|
|       Berk Goodwin|     Nigeria|
|      Palmer Burris| New Zealand|
|         Nehru Nash|     Belgium|
|      Kessie Branch|     Vietnam|
|     Emery Reynolds|     Ireland|
|      Buffy Coleman|      France|
|     Chava Roberson|     Ireland|
|    Maile Cervantes|        Peru|
|         Erin Frost|   Australia|
|     Jennifer Lucas| Philippines|
|Allistair Rodriguez|   Singapore|
|     Odysseus Mckay| Philippines|
+-------------------+------------+
only showing top 20 rows



Utilisation de la méthode filter
Dans cet exemple, nous allons selectionner tous les utilisateurs qui sont en france puis les compter.

In [13]:
users_csv_file.filter(users_csv_file['country'] == 'France').show()
number_of_users_in_france = users_csv_file.filter(users_csv_file['country'] == 'France').count()
print("Le nombre de user en france est de : ", number_of_users_in_france)

+---------------+--------------+-------+
|           name|         phone|country|
+---------------+--------------+-------+
|  Buffy Coleman|1-638-798-2347| France|
|  Jescie Vaughn|1-802-201-1155| France|
|Madeson Nichols|(872) 927-3473| France|
|Charlotte Boyer|(774) 724-7782| France|
|Maxwell Whitley|(322) 643-2583| France|
|  Gavin Pacheco|1-721-208-1874| France|
|     Hilda Soto|(419) 556-6647| France|
|    Maisie Mack|(685) 762-3116| France|
|    Otto Hansen|(445) 699-7373| France|
|  Cameron Reese|1-541-532-3151| France|
|  Stewart Cantu|(137) 676-2238| France|
+---------------+--------------+-------+

Le nombre de user en france est de :  11


Utilisation de plusieurs filtres
Dans cet exemple, nous allons:
  - selectionner tous les utilisateurs qui sont de France
  - puis afficher ceux qui ont la lettre y dans leur nom

In [14]:
users_csv_file.filter(users_csv_file['country'] == 'France').filter(users_csv_file['name'].contains("y")).show()

+---------------+--------------+-------+
|           name|         phone|country|
+---------------+--------------+-------+
|  Buffy Coleman|1-638-798-2347| France|
|Charlotte Boyer|(774) 724-7782| France|
|Maxwell Whitley|(322) 643-2583| France|
+---------------+--------------+-------+



Utilisation de la méthode Where.
Dans cet exemple, nous allons selectionner les utilisateurs qui sont en france et qui ont la lettre h dans leurs prenoms

In [18]:
users_csv_file.where((users_csv_file['country'] == 'France') & (users_csv_file['name'].contains('h'))).show()

+---------------+--------------+-------+
|           name|         phone|country|
+---------------+--------------+-------+
|  Jescie Vaughn|1-802-201-1155| France|
|Madeson Nichols|(872) 927-3473| France|
|Charlotte Boyer|(774) 724-7782| France|
|Maxwell Whitley|(322) 643-2583| France|
|  Gavin Pacheco|1-721-208-1874| France|
+---------------+--------------+-------+



Utilisation de groupBy, orderBy et count.
Dans cet exemple, nous allons: 
  - regrouper les utilisateurs par pays grace à groupBy()
  - puis les afficher par ordre décroissant grace à orderBy().

Il existe aussi de nombreux autres méthodes applicables telles que:
  - Mean : pour trouver la moyenne
  - Max : pour trouver le maximum
  - Min : pour trouver le minimum
  - Sum : pour faire la somme

In [24]:
for col in users_csv_file.columns:
    if(col != 'name'):
        print(f"Pour {col}")
        users_csv_file.groupBy('country').count().orderBy('count', ascending=False).show()

Pour phone
+------------------+-----+
|           country|count|
+------------------+-----+
|         Australia|   14|
|           Austria|   13|
|       Philippines|   13|
|           Germany|   12|
|            Mexico|   11|
|              Peru|   11|
|             India|   11|
|         Indonesia|   11|
|            France|   11|
|           Nigeria|   10|
|           Belgium|   10|
|Russian Federation|   10|
|           Ukraine|   10|
|    United Kingdom|   10|
|     United States|   10|
|       South Korea|   10|
|            Brazil|   10|
|             Spain|    9|
|            Canada|    9|
|      South Africa|    9|
+------------------+-----+
only showing top 20 rows

Pour country
+------------------+-----+
|           country|count|
+------------------+-----+
|         Australia|   14|
|           Austria|   13|
|       Philippines|   13|
|           Germany|   12|
|            Mexico|   11|
|              Peru|   11|
|             India|   11|
|         Indonesia|   11|
|    

 Utilisation de Collect.
    Cette méthode nous permet de transformer le dataframe en objet de type Collection.
    Dans cet exemple, nous allons utiliser groupBy() et collect_set() pour
        - regrouper les utilisateurs par pays
        - mettre tous les numeros de telephone du pays correspondant dans une collection

In [32]:
users_csv_file.groupBy('country').agg(F.collect_set('phone')).show()

+------------------+--------------------+
|           country|  collect_set(phone)|
+------------------+--------------------+
|            Sweden|[(739) 676-1865, ...|
|       Philippines|[1-743-338-2621, ...|
|         Singapore|[(311) 349-6570, ...|
|            Turkey|[1-843-647-5157, ...|
|           Germany|[1-756-662-7873, ...|
|            France|[1-638-798-2347, ...|
|           Belgium|[(768) 663-3445, ...|
|              Peru|[1-360-431-5265, ...|
|     United States|[(833) 886-5775, ...|
|             India|[(687) 787-2861, ...|
|             China|[1-615-305-5370, ...|
|             Chile|[(432) 354-8287, ...|
|           Nigeria|[1-713-118-4175, ...|
|             Italy|[1-575-351-1454, ...|
|            Norway|[(560) 871-7877, ...|
|             Spain|[1-224-872-1175, ...|
|Russian Federation|[1-576-698-2792, ...|
|           Ireland|[(153) 521-3522, ...|
|           Ukraine|[1-347-584-2442, ...|
|       South Korea|[1-868-578-5568, ...|
+------------------+--------------

In [40]:
# ajout d'une nouvelle colone avec une valeur constante
users_csv_file =  users_csv_file.withColumn('constant', F.lit('humain'))

In [41]:
users_csv_file.select(['name','constant']).show()

+-------------------+--------+
|               name|constant|
+-------------------+--------+
|       Adara Mcneil|  humain|
|     Barry Valencia|  humain|
|      Barry Delgado|  humain|
|      Shelley Oneal|  humain|
|     Wyoming Cherry|  humain|
|       Lani Holcomb|  humain|
|       Cedric Leach|  humain|
|          Edan Wood|  humain|
|       Berk Goodwin|  humain|
|      Palmer Burris|  humain|
|         Nehru Nash|  humain|
|      Kessie Branch|  humain|
|     Emery Reynolds|  humain|
|      Buffy Coleman|  humain|
|     Chava Roberson|  humain|
|    Maile Cervantes|  humain|
|         Erin Frost|  humain|
|     Jennifer Lucas|  humain|
|Allistair Rodriguez|  humain|
|     Odysseus Mckay|  humain|
+-------------------+--------+
only showing top 20 rows

